In [ ]:
import sys
import os
import numpy as np
import time
# Add the parent folder to sys.path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import density_generator
import numpy as np
import matplotlib.pyplot as plt
import qutip
import importlib
import helper_functions
import quantum_subroutines_pennylane
import pipeline_v200
import vqfe_subroutine

from random import randint
importlib.reload(density_generator)
importlib.reload(helper_functions)
importlib.reload(quantum_subroutines_pennylane)
importlib.reload(vqfe_subroutine)

pass;

In [ ]:
vqfe_subroutine.vqfe()

{'F_trunc': 0.9310884805213565,
 'F_star': 0.9999959735804583,
 'top_eigenvalues': array([0.93180756]),
 'trace_rho_m': 0.9318075604961024,
 'trace_sigma_m': 0.9303699554637759,
 'opt_params': array([[[-0.14742698, -0.00490705, -0.05175491],
         [-0.09279602, -0.01695435,  0.25232555]],
 
        [[ 0.04036903, -0.28092113,  0.96774771],
         [-0.0445241 ,  0.01688508,  0.02390345]]]),
 'rho_diag': array([[ 9.31807560e-01+5.55111512e-17j,  4.60349730e-04+1.56817702e-03j,
         -1.13167879e-04+1.13509805e-04j, -2.90350021e-05-1.27695227e-04j],
        [ 4.60349730e-04-1.56817702e-03j,  3.22245546e-02+0.00000000e+00j,
         -1.30996742e-06+1.92385479e-06j, -7.49367297e-05+1.12018366e-04j],
        [-1.13167879e-04-1.13509805e-04j, -1.30996742e-06-1.92385479e-06j,
          3.22145788e-02-5.42101086e-20j, -4.63574138e-04+5.93789067e-05j],
        [-2.90350021e-05+1.27695227e-04j, -7.49367297e-05-1.12018366e-04j,
         -4.63574138e-04-5.93789067e-05j,  3.75330610e-03-8.80

In [7]:
def example_demo():
    """
    Demonstration: 
    We'll pick n_qubits=1 for simplicity, 
    define two small density matrices for rho, rho_delta, 
    measure sub/super fidelities, and compute 
    approximate QFI bounds for a small delta=0.1.
    """
    n_qubits = 1

    # Example: a pure state for rho
    psi = np.array([1.0, 0.0])
    rho = np.outer(psi, psi)

    # Another pure state for rho_delta
    phi = np.array([np.sqrt(0.7), np.sqrt(0.3)])
    rho_delta = np.outer(phi, phi)

    # Measure sub- and super-fidelity
    E_val, R_val = quantum_subroutines_pennylane.measure_sub_super_fidelity(n_qubits, rho, rho_delta)
    print("Subfidelity E(rho, rho_delta) =", E_val)
    print("Superfidelity R(rho, rho_delta) =", R_val)

    # Bounds on the standard fidelity: sqrt(E) <= F <= sqrt(R).
    F_lower = np.sqrt(E_val)
    F_upper = np.sqrt(R_val)
    print("Lower bound on fidelity =", F_lower)
    print("Upper bound on fidelity =", F_upper)

    # For small parameter shift delta, eq. (3) in the paper gives:
    #   I_delta(rho) = (8 / delta^2)*(1 - F(rho, rho_delta)).
    # So using the sub- and super-fidelity as bounds on F,
    # we get bounds on the QFI:
    delta = 0.1
    I_lower = (8.0 / delta**2)*(1.0 - F_upper)
    I_upper = (8.0 / delta**2)*(1.0 - F_lower)
    print("Approx. Lower bound on QFI =", I_lower)
    print("Approx. Upper bound on QFI =", I_upper)

example_demo()

Subfidelity E(rho, rho_delta) = 0.6999999999999995
Superfidelity R(rho, rho_delta) = 0.7000000000000002
Lower bound on fidelity = 0.8366600265340752
Upper bound on fidelity = 0.8366600265340757
Approx. Lower bound on QFI = 130.67197877273944
Approx. Upper bound on QFI = 130.6719787727398


In [8]:
n = 5 ## numero finale di qubit
N = 7 ## numero di qubit prima della traccia parziale

h_z_s = np.linspace(0, 5, num=20) # spazio dei valori di h_z

a_x= 3

delta = 0.5
derivative_delta = 1e-5
m = 7
DEBUG=True
initial_state = helper_functions.random_mixed_density_matrix(N, n)

# ket = qutip.rand_ket(2**n, distribution="haar").full().flatten()

y_s = np.array([pipeline_v200.simulation(
                        N=N,
                        n=n,
                        initial_state= initial_state,
                        DEBUG=True,
                        derivative_delta=derivative_delta,
                        a_x= a_x, 
                        h_z= h_z, ## variable
                        delta = delta,
                        m = m) for h_z in h_z_s])

# just for plotting clarity
sparse_indices = np.array([i for i in list(range(len(h_z_s))) if i%10==0])

Generating random mixed density matrix
purity of initial state = (0.26981122154608717-8.74138001566438e-19j) 
Dimension of the Hilbert space: 32
Initial state type: <class 'numpy.ndarray'> | initial_state: [[ 0.02705162+4.75927742e-19j -0.0066634 -1.04648595e-03j
   0.00219118-8.08992609e-05j ...  0.00640547-3.48045042e-03j
   0.00103602-4.22397260e-03j  0.00293316+9.05987136e-03j]
 [-0.0066634 +1.04648595e-03j  0.03801386+5.42306155e-19j
  -0.00519149+1.57470705e-04j ...  0.004617  -4.57462584e-03j
  -0.00974733+1.88973953e-02j -0.00731824-5.04047077e-03j]
 [ 0.00219118+8.08992609e-05j -0.00519149-1.57470705e-04j
   0.01861122+1.05932863e-19j ... -0.00763026+4.78734553e-03j
   0.01822059-1.82303460e-02j -0.00737993-2.60657101e-03j]
 ...
 [ 0.00640547+3.48045042e-03j  0.004617  +4.57462584e-03j
  -0.00763026-4.78734553e-03j ...  0.01733077-1.36813895e-19j
  -0.01939275+5.62017750e-03j -0.01053872+3.33365784e-03j]
 [ 0.00103602+4.22397260e-03j -0.00974733-1.88973953e-02j
   0.01822059+1

purity of initial state = (0.26981122154608717-8.74138001566438e-19j) 
Dimension of the Hilbert space: 32
Initial state type: <class 'numpy.ndarray'> | initial_state: [[ 0.02705162+4.75927742e-19j -0.0066634 -1.04648595e-03j
   0.00219118-8.08992609e-05j ...  0.00640547-3.48045042e-03j
   0.00103602-4.22397260e-03j  0.00293316+9.05987136e-03j]
 [-0.0066634 +1.04648595e-03j  0.03801386+5.42306155e-19j
  -0.00519149+1.57470705e-04j ...  0.004617  -4.57462584e-03j
  -0.00974733+1.88973953e-02j -0.00731824-5.04047077e-03j]
 [ 0.00219118+8.08992609e-05j -0.00519149-1.57470705e-04j
   0.01861122+1.05932863e-19j ... -0.00763026+4.78734553e-03j
   0.01822059-1.82303460e-02j -0.00737993-2.60657101e-03j]
 ...
 [ 0.00640547+3.48045042e-03j  0.004617  +4.57462584e-03j
  -0.00763026-4.78734553e-03j ...  0.01733077-1.36813895e-19j
  -0.01939275+5.62017750e-03j -0.01053872+3.33365784e-03j]
 [ 0.00103602+4.22397260e-03j -0.00974733-1.88973953e-02j
   0.01822059+1.82303460e-02j ... -0.01939275-5.620177